In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
torch.__version__

'1.10.0+cu111'

# MNIST数据集手写数字识别

## 数据集介绍
MNIST包括6万张`28x28`的训练样本，1万张测试样本，很多教程都会对它”下手”几乎成为一个“典范”，可以说它就是计算机视觉里面的Hello World。所以我们这里也会使用MNIST来进行实战。

前面在介绍卷积神经网络的时候说到过LeNet-5，LeNet-5之所以强大就是因为在当时的环境下将MNIST数据的识别率提高到了99%，这里我们也自己从头搭建一个卷积神经网络，也达到99%的准确率

## 手写数字识别
首先，我们定义一些超参数

In [2]:
BATCH_SIZE=512 #大概需要2G的显存
EPOCHS=20 # 总共训练批次
print(torch.cuda.is_available())
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 让torch判断是否使用GPU，建议使用GPU环境，因为会快很多

True


因为Pytorch里面包含了MNIST的数据集，所以我们这里直接使用即可。
如果第一次执行会生成data文件夹，并且需要一些时间下载，如果以前下载过就不会再次下载了。

由于官方已经实现了dataset，所以这里可以直接使用DataLoader来对数据进行读取

In [3]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [4]:
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=BATCH_SIZE, shuffle=True)

下面我们定义一个网络，网络包含`两个卷积层`，`conv1`和`conv2`，然后紧接着`两个线性层`作为输出，最后`输出10个维度`，这10个维度我们作为0-9的标识来确定识别出的是那个数字

在这里建议大家将每一层的`输入和输出维度都作为注释标注`出来，这样后面阅读代码的会方便很多

In [5]:
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__() ## 继承nn.Module的__init___
    # batch*1*28*28（每次会送入batch个样本，输入通道数1（黑白图像），图像分辨率是28x28） ## weight * height * channel
    # 下面的卷积层Conv2d的第一个参数指输入通道数，第二个参数指输出通道数，第三个参数指卷积核的大小
    self.conv1 = nn.Conv2d(1,10,5) ## 1个input_channel 10个output_channel 5*5 filter
    self.conv2 = nn.Conv2d(10,20,3) 
    # 下面的全连接层Linear的第一个参数指输入通道数，第二个参数指输出通道数
    self.fc1 = nn.Linear(20*10*10,500) 
    self.fc2 = nn.Linear(500,10) ## fc1输出500 channel，最后10分类，所以500,10
  def forward(self,x):
    in_size=x.size(0) ## 前面设置了batch size=512，因此一次输入的实例数就是512，相当于512*1*28*28（实例数*通道数*宽*高）的tensor
    ## .size(0)，行数
    out=self.conv1(x) ## 卷积层1, input 512*1*28*28 -> 28-5+1=24 -> 512*10*24*24
    out=F.relu(out) ## ReLu激活
    out=F.max_pool2d(out,2) ## 2*2 池化层，如果池化层是正方形，可以用一个数字n来代替n*n, 512*10*24*24 -> (24-2)/2+1=12 -> 512*10*12*12
    out=self.conv2(out) ## 卷积层2 ## 512*10*12*12 -> 12-3+1=10 -> 512*20*10*10
    out=F.relu(out) ## 激活
    out=out.view(in_size,-1) ## 换成512行（实例），20*10*10（每个实例的特征）
    out=self.fc1(out) ## 输入通道数 20*10*10 （卷积后的结果换成一列）
    out=F.relu(out) 
    out=self.fc2(out) ## FC1输出通道数500
    out=F.log_softmax(out,dim=1)
    return out

我们实例化一个网络，实例化后使用.to方法将网络移动到GPU

优化器我们也直接选择简单暴力的Adam

In [11]:
model = ConvNet().to(DEVICE) ## 网络移到GPU
optimizer = optim.Adam(model.parameters()) ## Adam优化器

下面定义一下训练的函数，我们将训练的所有操作都封装到这个函数中

In [12]:
def train(model, device, train_loader, optimizer, epoch):
    model.train() ## 模型训练，一般放在循环外面
    ## 启用 Batch Normalization 和 Dropout。
    ## 如果模型中有BN层(Batch Normalization）和 Dropout，需要在训练时添加model.train()。
    ## model.train()是保证BN层能够用到每一批数据的均值和方差。
    ## 对于Dropout，model.train()是随机取一部分网络连接来训练更新参数。

    for batch_idx, (data, target) in enumerate(train_loader): ## 设置在Epoch中循环
        data, target = data.to(device), target.to(device) ## 数据移到GPU中
        optimizer.zero_grad() ## 优化器梯度归零
        output = model(data) ## 计算h(θ)
        loss = F.nll_loss(output, target) ## 计算loss
        loss.backward() ## 反向传播
        optimizer.step() ## 优化器更新权重
        if(batch_idx+1)%30 == 0:  ## 每30个Epoch输出一次loss
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

测试的操作也一样封装成一个函数

In [13]:
def test(model, device, test_loader):
    model.eval() 
    ## 不启用 Batch Normalization 和 Dropout。
    ## 如果模型中有BN层(Batch Normalization）和Dropout，在测试时添加model.eval()。
    ## model.eval()是保证BN层能够用全部训练数据的均值和方差，即测试过程中要保证BN层的均值和方差不变。
    ## 对于Dropout，model.eval()是利用到了所有网络连接，即不进行随机舍弃神经元。

    ## 训练完train样本后，生成的模型model要用来测试样本。
    ## 在model(test)之前，需要加上model.eval()，否则的话，有输入数据，即使不训练，它也会改变权值。这是model中含有BN层和Dropout所带来的的性质。

    ## model.train()和model.eval()相当于是一种声明，在Batch normalization和Dropout存在的情况下，如果是在test set或是validation set中进行模型预测的话，就不要采用BN和Dropout
    test_loss = 0 
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # 将一批的损失相加
            pred = output.max(1, keepdim=True)[1] # 找到概率最大的下标
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

下面开始训练，这里就体现出封装起来的好处了，只要写两行就可以了

In [14]:
for epoch in range(1, EPOCHS + 1):
    train(model, DEVICE, train_loader, optimizer, epoch)
    test(model, DEVICE, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.258221
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.201263
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.128812

Test set: Average loss: 0.1003, Accuracy: 9704/10000 (97%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.087452
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.066395
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.069093

Test set: Average loss: 0.0623, Accuracy: 9816/10000 (98%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.058434
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.066959
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.054916

Test set: Average loss: 0.0526, Accuracy: 9839/10000 (98%)

Train Epoch: 4 [14848/60000 (25%)]	Loss: 0.036556
Train Epoch: 4 [30208/60000 (50%)]	Loss: 0.036246
Train Epoch: 4 [45568/60000 (75%)]	Loss: 0.034024

Test set: Average loss: 0.0496, Accuracy: 9838/10000 (98%)

Train Epoch: 5 [14848/60000 (25%)]	Loss: 0.075570
Train Epoch: 5 [30208/60000 (50%)]	Loss: 0.035958
Train Epoch: 5 [45568/60000 (75%)]	Loss: 0.028164

Test 

如果你的模型连MNIST都搞不定，那么你的模型没有任何的价值

即使你的模型搞定了MNIST，你的模型也可能没有任何的价值

MNIST是一个很简单的数据集，由于它的局限性只能作为研究用途，对实际应用带来的价值非常有限。但是通过这个例子，我们可以完全了解一个实际项目的工作流程

我们找到数据集，对数据做预处理，定义我们的模型，调整超参数，测试训练，再通过训练结果对超参数进行调整或者对模型进行调整。

并且通过这个实战我们已经有了一个很好的模板，以后的项目都可以以这个模板为样例